# Import libraries and data

In [1]:
# Import libraries
import os
import openai
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Get OpenAI API secret key (change this file with your own OpenAI key)
open_ai_key = open('../data/local/openai_key.txt', 'r').read()
openai.api_key = open_ai_key
#openai.Model.list()

In [3]:
# Prompt for GPT
prompt_raw = "I would like to summarise in maximum 300 characters and in English what the following sentences in Portuguese talk about. \
Do not use aspects mentioned only in one sentence as part of the summary: TWEETS"

# 2013 negative

## January

In [4]:
# Get monthly data
df_2013_01_neg = pd.read_csv('../data/local/df_neg_2013-01.csv').reset_index()
df_2013_01_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1327 entries, 0 to 1326
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1327 non-null   int64  
 1   LocationCode            1327 non-null   object 
 2   LocationName            1327 non-null   object 
 3   StatesCode              1327 non-null   object 
 4   ...4                    1327 non-null   int64  
 5   id                      1327 non-null   float64
 6   text                    1327 non-null   object 
 7   label                   1327 non-null   object 
 8   in_reply_to_status_id   97 non-null     float64
 9   in_reply_to_user_id     104 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       189 non-null    float64
 13  retweeted_status_id     189 non-null    float64
 14  created_at              1327 non-null   

### Representative sample based on week and location

In [5]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_01_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_01_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w01,Amazonas,4,0.003014
1,2013-w01,Bahia,6,0.004521
2,2013-w01,Espirito Santo,1,0.000754
3,2013-w01,Goias,1,0.000754
4,2013-w01,Maranhao,4,0.003014
...,...,...,...,...
88,2013-w05,Rio Grande Do Sul,13,0.009797
89,2013-w05,Roraima,1,0.000754
90,2013-w05,Santa Catarina,6,0.004521
91,2013-w05,Sao Paulo,47,0.035418


In [6]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w01,Amazonas,4,0.003014,7
1,2013-w01,Bahia,6,0.004521,11
2,2013-w01,Espirito Santo,1,0.000754,1
3,2013-w01,Goias,1,0.000754,1
4,2013-w01,Maranhao,4,0.003014,7
...,...,...,...,...,...
88,2013-w05,Rio Grande Do Sul,13,0.009797,24
89,2013-w05,Roraima,1,0.000754,1
90,2013-w05,Santa Catarina,6,0.004521,11
91,2013-w05,Sao Paulo,47,0.035418,86


In [7]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_01_neg_sample_size = pd.merge(df_2013_01_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [8]:
# Step 6: Perform the stratified sampling
#df_2013_01_neg_sample = df_2013_01_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_01_neg_sample = df_2013_01_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_01_neg_sample)} rows.")


Sampled 1327 rows.


In [9]:
text_gpt_2013_01_neg = '\n'.join(df_2013_01_neg_sample['text'].astype(str))

### Prompts and loops

In [10]:
prompt_1 = prompt_raw.replace('TWEETS', text_gpt_2013_01_neg)

In [11]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_1}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [12]:
response.choices[0].message.content

'The texts discuss various vaccine-related issues, including concerns about the safety and side effects of certain vaccines, such as the H1N1 vaccine potentially causing narcolepsy in children. There\'s also mention of a controversial "frog vaccine" under investigation by federal authorities, and reports of vaccine-induced severe reactions, including an incurable sleeping disease affecting hundreds of youths. Additionally, there are personal anecdotes expressing fear and pain associated with vaccinations, particularly for the HPV and Tetanus vaccines, highlighting a societal aversion and anxiety towards vaccine administration.'

In [13]:
summary = []
dataset = []

In [14]:
dataset.append("2013_01_negative")

In [15]:
summary.append(response.choices[0].message.content)

In [16]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  1 non-null      object
 1   summary  1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [17]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...


In [18]:
response

<OpenAIObject chat.completion id=chatcmpl-AUehhDM8DSBunyUWP5pLXUEzvHzri at 0x249ea624e50> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The texts discuss various vaccine-related issues, including concerns about the safety and side effects of certain vaccines, such as the H1N1 vaccine potentially causing narcolepsy in children. There's also mention of a controversial \"frog vaccine\" under investigation by federal authorities, and reports of vaccine-induced severe reactions, including an incurable sleeping disease affecting hundreds of youths. Additionally, there are personal anecdotes expressing fear and pain associated with vaccinations, particularly for the HPV and Tetanus vaccines, highlighting a societal aversion and anxiety towards vaccine administration.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869669,
  "id": "chatcmpl-AUehhDM8DSBunyUWP

## February

In [19]:
# Get monthly data
df_2013_02_neg = pd.read_csv('../data/local/df_neg_2013-02.csv').reset_index()
df_2013_02_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   197 non-null    int64  
 1   LocationCode            197 non-null    object 
 2   LocationName            197 non-null    object 
 3   StatesCode              197 non-null    object 
 4   ...4                    197 non-null    int64  
 5   id                      197 non-null    float64
 6   text                    197 non-null    object 
 7   label                   197 non-null    object 
 8   in_reply_to_status_id   21 non-null     float64
 9   in_reply_to_user_id     25 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       12 non-null     float64
 13  retweeted_status_id     12 non-null     float64
 14  created_at              197 non-null    ob

### Representative sample based on week and location

In [20]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_02_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_02_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w05,Alagoas,2,0.010152
1,2013-w05,Amazonas,3,0.015228
2,2013-w05,Bahia,4,0.020305
3,2013-w05,Ceara,3,0.015228
4,2013-w05,Distrito Federal,5,0.025381
5,2013-w05,Espirito Santo,4,0.020305
6,2013-w05,Mato Grosso,2,0.010152
7,2013-w05,Minas Gerais,28,0.142132
8,2013-w05,Paraiba,2,0.010152
9,2013-w05,Parana,5,0.025381


In [21]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w05,Alagoas,2,0.010152,24
1,2013-w05,Amazonas,3,0.015228,37
2,2013-w05,Bahia,4,0.020305,49
3,2013-w05,Ceara,3,0.015228,37
4,2013-w05,Distrito Federal,5,0.025381,62
5,2013-w05,Espirito Santo,4,0.020305,49
6,2013-w05,Mato Grosso,2,0.010152,24
7,2013-w05,Minas Gerais,28,0.142132,348
8,2013-w05,Paraiba,2,0.010152,24
9,2013-w05,Parana,5,0.025381,62


In [22]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_02_neg_sample_size = pd.merge(df_2013_02_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [23]:
# Step 6: Perform the stratified sampling
#df_2013_02_neg_sample = df_2013_02_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_02_neg_sample = df_2013_02_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_02_neg_sample)} rows.")


Sampled 197 rows.


In [24]:
text_gpt_2013_02_neg = '\n'.join(df_2013_02_neg_sample['text'].astype(str))

### Prompts and loops

In [25]:
prompt_2 = prompt_raw.replace('TWEETS', text_gpt_2013_02_neg)

In [26]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_2}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [27]:
response.choices[0].message.content

'The text discusses various experiences and opinions regarding vaccinations, highlighting fears, reactions, and frustrations associated with getting vaccinated. It mentions a false nurse kidnapping a newborn under the pretext of vaccination and includes personal anecdotes about pain, side effects, and the anxiety surrounding vaccine appointments.'

In [28]:
#summary = []
#dataset = []

In [29]:
dataset.append("2013_02_negative")

In [30]:
summary.append(response.choices[0].message.content)

In [31]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  2 non-null      object
 1   summary  2 non-null      object
dtypes: object(2)
memory usage: 160.0+ bytes


In [32]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...


In [33]:
response

<OpenAIObject chat.completion id=chatcmpl-AUehklVlIe4Ag6WScRUe9VmWpZpcB at 0x249ea6562c0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various experiences and opinions regarding vaccinations, highlighting fears, reactions, and frustrations associated with getting vaccinated. It mentions a false nurse kidnapping a newborn under the pretext of vaccination and includes personal anecdotes about pain, side effects, and the anxiety surrounding vaccine appointments.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869672,
  "id": "chatcmpl-AUehklVlIe4Ag6WScRUe9VmWpZpcB",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 53,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_token

## March

In [34]:
# Get monthly data
df_2013_03_neg = pd.read_csv('../data/local/df_neg_2013-03.csv').reset_index()
df_2013_03_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   110 non-null    int64  
 1   LocationCode            110 non-null    object 
 2   LocationName            110 non-null    object 
 3   StatesCode              110 non-null    object 
 4   ...4                    110 non-null    int64  
 5   id                      110 non-null    float64
 6   text                    110 non-null    object 
 7   label                   110 non-null    object 
 8   in_reply_to_status_id   12 non-null     float64
 9   in_reply_to_user_id     13 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       0 non-null      float64
 13  retweeted_status_id     0 non-null      float64
 14  created_at              110 non-null    ob

### Representative sample based on week and location

In [35]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_03_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_03_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w10,Minas Gerais,1,0.009091
1,2013-w13,Amazonas,2,0.018182
2,2013-w13,Bahia,9,0.081818
3,2013-w13,Ceara,2,0.018182
4,2013-w13,Distrito Federal,5,0.045455
5,2013-w13,Espirito Santo,2,0.018182
6,2013-w13,Goias,1,0.009091
7,2013-w13,Mato Grosso Do Sul,4,0.036364
8,2013-w13,Minas Gerais,19,0.172727
9,2013-w13,Paraiba,3,0.027273


In [36]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w10,Minas Gerais,1,0.009091,22
1,2013-w13,Amazonas,2,0.018182,44
2,2013-w13,Bahia,9,0.081818,200
3,2013-w13,Ceara,2,0.018182,44
4,2013-w13,Distrito Federal,5,0.045455,111
5,2013-w13,Espirito Santo,2,0.018182,44
6,2013-w13,Goias,1,0.009091,22
7,2013-w13,Mato Grosso Do Sul,4,0.036364,89
8,2013-w13,Minas Gerais,19,0.172727,423
9,2013-w13,Paraiba,3,0.027273,66


In [37]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_03_neg_sample_size = pd.merge(df_2013_03_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [38]:
# Step 6: Perform the stratified sampling
#df_2013_03_neg_sample = df_2013_03_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_03_neg_sample = df_2013_03_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_03_neg_sample)} rows.")


Sampled 110 rows.


In [39]:
text_gpt_2013_03_neg = '\n'.join(df_2013_03_neg_sample['text'].astype(str))

### Prompts and loops

In [40]:
prompt_3 = prompt_raw.replace('TWEETS', text_gpt_2013_03_neg)

In [41]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_3}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [42]:
response.choices[0].message.content

'The text expresses frustration and concerns about vaccines in the 21st century, including the lack of a vaccine for acne, pain and side effects from vaccinations, and past controversies regarding vaccines. It highlights mixed feelings toward vaccines, with some people fearing injections while others share experiences of discomfort or adverse reactions after receiving them.'

In [43]:
#summary = []
#dataset = []

In [44]:
dataset.append("2013_03_negative")

In [45]:
summary.append(response.choices[0].message.content)

In [46]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  3 non-null      object
 1   summary  3 non-null      object
dtypes: object(2)
memory usage: 176.0+ bytes


In [47]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...


In [48]:
response

<OpenAIObject chat.completion id=chatcmpl-AUehl59VSquKYeoqJnlixkNdNac0t at 0x249ea71fd60> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text expresses frustration and concerns about vaccines in the 21st century, including the lack of a vaccine for acne, pain and side effects from vaccinations, and past controversies regarding vaccines. It highlights mixed feelings toward vaccines, with some people fearing injections while others share experiences of discomfort or adverse reactions after receiving them.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869673,
  "id": "chatcmpl-AUehl59VSquKYeoqJnlixkNdNac0t",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 62,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens

## April

In [49]:
# Get monthly data
df_2013_04_neg = pd.read_csv('../data/local/df_neg_2013-04.csv').reset_index()
df_2013_04_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2784 entries, 0 to 2783
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2784 non-null   int64  
 1   LocationCode            2784 non-null   object 
 2   LocationName            2784 non-null   object 
 3   StatesCode              2784 non-null   object 
 4   ...4                    2784 non-null   int64  
 5   id                      2784 non-null   float64
 6   text                    2784 non-null   object 
 7   label                   2784 non-null   object 
 8   in_reply_to_status_id   222 non-null    float64
 9   in_reply_to_user_id     266 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       0 non-null      float64
 13  retweeted_status_id     0 non-null      float64
 14  created_at              2784 non-null   

### Representative sample based on week and location

In [50]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_04_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_04_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w13,Ceara,1,0.000359
1,2013-w13,Distrito Federal,1,0.000359
2,2013-w13,Goias,1,0.000359
3,2013-w13,Minas Gerais,9,0.003233
4,2013-w13,Paraiba,1,0.000359
...,...,...,...,...
108,2013-w18,Parana,6,0.002155
109,2013-w18,Rio Grande Do Sul,24,0.008621
110,2013-w18,Santa Catarina,11,0.003951
111,2013-w18,Sao Paulo,51,0.018319


In [51]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w13,Ceara,1,0.000359,0
1,2013-w13,Distrito Federal,1,0.000359,0
2,2013-w13,Goias,1,0.000359,0
3,2013-w13,Minas Gerais,9,0.003233,7
4,2013-w13,Paraiba,1,0.000359,0
...,...,...,...,...,...
108,2013-w18,Parana,6,0.002155,5
109,2013-w18,Rio Grande Do Sul,24,0.008621,21
110,2013-w18,Santa Catarina,11,0.003951,9
111,2013-w18,Sao Paulo,51,0.018319,44


In [52]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_04_neg_sample_size = pd.merge(df_2013_04_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [53]:
# Step 6: Perform the stratified sampling
#df_2013_04_neg_sample = df_2013_04_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_04_neg_sample = df_2013_04_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_04_neg_sample)} rows.")


Sampled 2784 rows.


In [54]:
text_gpt_2013_04_neg = '\n'.join(df_2013_04_neg_sample['text'].astype(str))

### Prompts and loops

In [55]:
prompt_4 = prompt_raw.replace('TWEETS', text_gpt_2013_04_neg)

In [56]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_4}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [57]:
response.choices[0].message.content

'The texts express a mix of humor, fear, and frustration surrounding vaccinations, particularly the flu vaccine and the controversial "frog vaccine." Many share experiences of pain from the shots, express reluctance to get vaccinated, and criticize the disparity in access to vaccines, especially for prisoners compared to the general public. There are also mentions of adverse reactions, anxieties about needles, and the perception of vaccines as ineffective or harmful. Overall, the sentiment conveys a societal struggle with vaccination practices and personal discomfort related to them.'

In [58]:
#summary = []
#dataset = []

In [59]:
dataset.append("2013_04_negative")

In [60]:
summary.append(response.choices[0].message.content)

In [61]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  4 non-null      object
 1   summary  4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


In [62]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."


In [63]:
response

<OpenAIObject chat.completion id=chatcmpl-AUehpqGeHD7IOrJYzPTTznCwmKkU7 at 0x249eb704b80> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The texts express a mix of humor, fear, and frustration surrounding vaccinations, particularly the flu vaccine and the controversial \"frog vaccine.\" Many share experiences of pain from the shots, express reluctance to get vaccinated, and criticize the disparity in access to vaccines, especially for prisoners compared to the general public. There are also mentions of adverse reactions, anxieties about needles, and the perception of vaccines as ineffective or harmful. Overall, the sentiment conveys a societal struggle with vaccination practices and personal discomfort related to them.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869677,
  "id": "chatcmpl-AUehpqGeHD7IOrJYzPTTznCwmKkU7",
  "model": "gpt-4o-mini-2024

## May

In [64]:
# Get monthly data
df_2013_05_neg = pd.read_csv('../data/local/df_neg_2013-05.csv').reset_index()
df_2013_05_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   2542 non-null   int64  
 1   LocationCode            2542 non-null   object 
 2   LocationName            2542 non-null   object 
 3   StatesCode              2542 non-null   object 
 4   ...4                    2542 non-null   int64  
 5   id                      2542 non-null   float64
 6   text                    2542 non-null   object 
 7   label                   2542 non-null   object 
 8   in_reply_to_status_id   220 non-null    float64
 9   in_reply_to_user_id     241 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       0 non-null      float64
 13  retweeted_status_id     0 non-null      float64
 14  created_at              2542 non-null   

### Representative sample based on week and location

In [65]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_05_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_05_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w18,Amapa,4,0.001574
1,2013-w18,Amazonas,7,0.002754
2,2013-w18,Bahia,21,0.008261
3,2013-w18,Ceara,7,0.002754
4,2013-w18,Distrito Federal,7,0.002754
...,...,...,...,...
101,2013-w22,Pernambuco,2,0.000787
102,2013-w22,Rio Grande Do Sul,29,0.011408
103,2013-w22,Santa Catarina,14,0.005507
104,2013-w22,Sao Paulo,136,0.053501


In [66]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w18,Amapa,4,0.001574,3
1,2013-w18,Amazonas,7,0.002754,6
2,2013-w18,Bahia,21,0.008261,20
3,2013-w18,Ceara,7,0.002754,6
4,2013-w18,Distrito Federal,7,0.002754,6
...,...,...,...,...,...
101,2013-w22,Pernambuco,2,0.000787,1
102,2013-w22,Rio Grande Do Sul,29,0.011408,27
103,2013-w22,Santa Catarina,14,0.005507,13
104,2013-w22,Sao Paulo,136,0.053501,131


In [67]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_05_neg_sample_size = pd.merge(df_2013_05_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [68]:
# Step 6: Perform the stratified sampling
#df_2013_05_neg_sample = df_2013_05_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_05_neg_sample = df_2013_05_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_05_neg_sample)} rows.")


Sampled 2542 rows.


In [69]:
text_gpt_2013_05_neg = '\n'.join(df_2013_05_neg_sample['text'].astype(str))

### Prompts and loops

In [70]:
prompt_5 = prompt_raw.replace('TWEETS', text_gpt_2013_05_neg)

In [71]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_5}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [72]:
response.choices[0].message.content

'The sentences express a widespread dislike and fear associated with vaccinations, particularly against the flu and for various childhood immunizations. Many individuals report experiencing pain, discomfort, and adverse reactions after getting vaccinated, such as feeling sick or developing symptoms of illness. There are humorous and sarcastic remarks about the efficacy of vaccines, with some people feeling frustrated that they still get sick despite being vaccinated. The mention of reactions to vaccines, such as fever or soreness, and the reluctance to receive them highlights a common sentiment of apprehension and distrust towards vaccination processes.'

In [73]:
#summary = []
#dataset = []

In [74]:
dataset.append("2013_05_negative")

In [75]:
summary.append(response.choices[0].message.content)

In [76]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  5 non-null      object
 1   summary  5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [77]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...


In [78]:
response

<OpenAIObject chat.completion id=chatcmpl-AUehyz5I7jBzFnApUhIky5q9iZUaw at 0x249ebb7b770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences express a widespread dislike and fear associated with vaccinations, particularly against the flu and for various childhood immunizations. Many individuals report experiencing pain, discomfort, and adverse reactions after getting vaccinated, such as feeling sick or developing symptoms of illness. There are humorous and sarcastic remarks about the efficacy of vaccines, with some people feeling frustrated that they still get sick despite being vaccinated. The mention of reactions to vaccines, such as fever or soreness, and the reluctance to receive them highlights a common sentiment of apprehension and distrust towards vaccination processes.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869686,
  "id": "

## June

In [79]:
# Get monthly data
df_2013_06_neg = pd.read_csv('../data/local/df_neg_2013-06.csv').reset_index()
df_2013_06_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1959 entries, 0 to 1958
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1959 non-null   int64  
 1   LocationCode            1959 non-null   object 
 2   LocationName            1959 non-null   object 
 3   StatesCode              1959 non-null   object 
 4   ...4                    1959 non-null   int64  
 5   id                      1959 non-null   float64
 6   text                    1959 non-null   object 
 7   label                   1959 non-null   object 
 8   in_reply_to_status_id   188 non-null    float64
 9   in_reply_to_user_id     196 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       335 non-null    float64
 13  retweeted_status_id     335 non-null    float64
 14  created_at              1959 non-null   

### Representative sample based on week and location

In [80]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_06_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_06_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w22,Amazonas,2,0.001021
1,2013-w22,Bahia,9,0.004594
2,2013-w22,Ceara,2,0.001021
3,2013-w22,Distrito Federal,3,0.001531
4,2013-w22,Maranhao,7,0.003573
...,...,...,...,...
96,2013-w26,Rio De Janeiro,1,0.000510
97,2013-w26,Rio Grande Do Sul,54,0.027565
98,2013-w26,Roraima,1,0.000510
99,2013-w26,Santa Catarina,8,0.004084


In [81]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w22,Amazonas,2,0.001021,2
1,2013-w22,Bahia,9,0.004594,11
2,2013-w22,Ceara,2,0.001021,2
3,2013-w22,Distrito Federal,3,0.001531,3
4,2013-w22,Maranhao,7,0.003573,8
...,...,...,...,...,...
96,2013-w26,Rio De Janeiro,1,0.000510,1
97,2013-w26,Rio Grande Do Sul,54,0.027565,67
98,2013-w26,Roraima,1,0.000510,1
99,2013-w26,Santa Catarina,8,0.004084,10


In [82]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_06_neg_sample_size = pd.merge(df_2013_06_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [83]:
# Step 6: Perform the stratified sampling
#df_2013_06_neg_sample = df_2013_06_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_06_neg_sample = df_2013_06_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_06_neg_sample)} rows.")


Sampled 1959 rows.


In [84]:
text_gpt_2013_06_neg = '\n'.join(df_2013_06_neg_sample['text'].astype(str))

### Prompts and loops

In [85]:
prompt_6 = prompt_raw.replace('TWEETS', text_gpt_2013_06_neg)

In [86]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_6}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [87]:
response.choices[0].message.content

'The text primarily discusses the controversial topic of a "gay cure vaccine," often portrayed humorously or sarcastically. Many comments express disdain or skepticism about the concept, with repeated statements about only accepting the vaccine if it involves a large needle. There are also complaints about real vaccines, like those for flu and other diseases, highlighting physical discomfort and frustrations with public health services. Overall, it reflects a mix of humor, sarcasm, and genuine concern regarding health and vaccination practices.'

In [88]:
#summary = []
#dataset = []

In [89]:
dataset.append("2013_06_negative")

In [90]:
summary.append(response.choices[0].message.content)

In [91]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  6 non-null      object
 1   summary  6 non-null      object
dtypes: object(2)
memory usage: 224.0+ bytes


In [92]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...


In [93]:
response

<OpenAIObject chat.completion id=chatcmpl-AUei5HLWh1w9SiFOkmvm4Qb3kNBrk at 0x249ebd1f590> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text primarily discusses the controversial topic of a \"gay cure vaccine,\" often portrayed humorously or sarcastically. Many comments express disdain or skepticism about the concept, with repeated statements about only accepting the vaccine if it involves a large needle. There are also complaints about real vaccines, like those for flu and other diseases, highlighting physical discomfort and frustrations with public health services. Overall, it reflects a mix of humor, sarcasm, and genuine concern regarding health and vaccination practices.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869693,
  "id": "chatcmpl-AUei5HLWh1w9SiFOkmvm4Qb3kNBrk",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",


## July

In [94]:
# Get monthly data
df_2013_07_neg = pd.read_csv('../data/local/df_neg_2013-07.csv').reset_index()
df_2013_07_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1405 entries, 0 to 1404
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1405 non-null   int64  
 1   LocationCode            1405 non-null   object 
 2   LocationName            1405 non-null   object 
 3   StatesCode              1405 non-null   object 
 4   ...4                    1405 non-null   int64  
 5   id                      1405 non-null   float64
 6   text                    1405 non-null   object 
 7   label                   1405 non-null   object 
 8   in_reply_to_status_id   120 non-null    float64
 9   in_reply_to_user_id     130 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       151 non-null    float64
 13  retweeted_status_id     151 non-null    float64
 14  created_at              1405 non-null   

### Representative sample based on week and location

In [95]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_07_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_07_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w26,Amazonas,2,0.001423
1,2013-w26,Distrito Federal,2,0.001423
2,2013-w26,Espirito Santo,1,0.000712
3,2013-w26,Maranhao,2,0.001423
4,2013-w26,Mato Grosso,1,0.000712
...,...,...,...,...
105,2013-w31,Rio De Janeiro,2,0.001423
106,2013-w31,Rio Grande Do Sul,9,0.006406
107,2013-w31,Santa Catarina,4,0.002847
108,2013-w31,Sao Paulo,30,0.021352


In [96]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w26,Amazonas,2,0.001423,3
1,2013-w26,Distrito Federal,2,0.001423,3
2,2013-w26,Espirito Santo,1,0.000712,1
3,2013-w26,Maranhao,2,0.001423,3
4,2013-w26,Mato Grosso,1,0.000712,1
...,...,...,...,...,...
105,2013-w31,Rio De Janeiro,2,0.001423,3
106,2013-w31,Rio Grande Do Sul,9,0.006406,15
107,2013-w31,Santa Catarina,4,0.002847,6
108,2013-w31,Sao Paulo,30,0.021352,52


In [97]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_07_neg_sample_size = pd.merge(df_2013_07_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [98]:
# Step 6: Perform the stratified sampling
#df_2013_07_neg_sample = df_2013_07_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_07_neg_sample = df_2013_07_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_07_neg_sample)} rows.")


Sampled 1405 rows.


In [99]:
text_gpt_2013_07_neg = '\n'.join(df_2013_07_neg_sample['text'].astype(str))

### Prompts and loops

In [100]:
prompt_7 = prompt_raw.replace('TWEETS', text_gpt_2013_07_neg)

In [101]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_7}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [102]:
response.choices[0].message.content

"The sentences discuss various opinions and experiences regarding vaccinations, particularly focusing on pain and discomfort associated with them. Many express frustration about vaccines, stating they are painful and sometimes ineffective, like the flu vaccine. There's also humor relating to the absurdity of certain vaccines, such as the HPV vaccine for young girls, and societal attitudes towards vaccinations. Overall, there's a prevalent theme of reluctance and anxiety surrounding the vaccination process."

In [103]:
#summary = []
#dataset = []

In [104]:
dataset.append("2013_07_negative")

In [105]:
summary.append(response.choices[0].message.content)

In [106]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  7 non-null      object
 1   summary  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [107]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...
6,2013_07_negative,The sentences discuss various opinions and exp...


In [108]:
response

<OpenAIObject chat.completion id=chatcmpl-AUeiBEv8ZGQfFhkrW1uVBM7Mnm4qM at 0x249ebe6a8b0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The sentences discuss various opinions and experiences regarding vaccinations, particularly focusing on pain and discomfort associated with them. Many express frustration about vaccines, stating they are painful and sometimes ineffective, like the flu vaccine. There's also humor relating to the absurdity of certain vaccines, such as the HPV vaccine for young girls, and societal attitudes towards vaccinations. Overall, there's a prevalent theme of reluctance and anxiety surrounding the vaccination process.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869699,
  "id": "chatcmpl-AUeiBEv8ZGQfFhkrW1uVBM7Mnm4qM",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",


## August

In [109]:
# Get monthly data
df_2013_08_neg = pd.read_csv('../data/local/df_neg_2013-08.csv').reset_index()
df_2013_08_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154 entries, 0 to 1153
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   1154 non-null   int64  
 1   LocationCode            1154 non-null   object 
 2   LocationName            1154 non-null   object 
 3   StatesCode              1154 non-null   object 
 4   ...4                    1154 non-null   int64  
 5   id                      1154 non-null   float64
 6   text                    1154 non-null   object 
 7   label                   1154 non-null   object 
 8   in_reply_to_status_id   107 non-null    float64
 9   in_reply_to_user_id     116 non-null    float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       98 non-null     float64
 13  retweeted_status_id     98 non-null     float64
 14  created_at              1154 non-null   

### Representative sample based on week and location

In [110]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_08_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_08_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w31,Acre,1,0.000867
1,2013-w31,Amazonas,3,0.002600
2,2013-w31,Bahia,9,0.007799
3,2013-w31,Ceara,2,0.001733
4,2013-w31,Distrito Federal,2,0.001733
...,...,...,...,...
93,2013-w35,Pernambuco,1,0.000867
94,2013-w35,Piaui,1,0.000867
95,2013-w35,Rio Grande Do Sul,12,0.010399
96,2013-w35,Santa Catarina,2,0.001733


In [111]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w31,Acre,1,0.000867,2
1,2013-w31,Amazonas,3,0.002600,6
2,2013-w31,Bahia,9,0.007799,19
3,2013-w31,Ceara,2,0.001733,4
4,2013-w31,Distrito Federal,2,0.001733,4
...,...,...,...,...,...
93,2013-w35,Pernambuco,1,0.000867,2
94,2013-w35,Piaui,1,0.000867,2
95,2013-w35,Rio Grande Do Sul,12,0.010399,25
96,2013-w35,Santa Catarina,2,0.001733,4


In [112]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_08_neg_sample_size = pd.merge(df_2013_08_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [113]:
# Step 6: Perform the stratified sampling
#df_2013_08_neg_sample = df_2013_08_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_08_neg_sample = df_2013_08_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_08_neg_sample)} rows.")


Sampled 1154 rows.


In [114]:
text_gpt_2013_08_neg = '\n'.join(df_2013_08_neg_sample['text'].astype(str))

### Prompts and loops

In [115]:
prompt_8 = prompt_raw.replace('TWEETS', text_gpt_2013_08_neg)

In [116]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_8}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [117]:
response.choices[0].message.content

"The text discusses various opinions and experiences regarding vaccinations in Brazil, including pain and discomfort associated with certain vaccines, especially the HPV and tetanus vaccines. Many express frustration with the healthcare system's inefficiencies, lack of available vaccines, and the feeling of being forced to vaccinate. There's also mention of new vaccines being developed or tested, including one against cancer and another for HIV, alongside criticism of government policies related to vaccination campaigns. Overall, the sentiment reflects a mix of fear, skepticism, and dissatisfaction with vaccination processes."

In [118]:
#summary = []
#dataset = []

In [119]:
dataset.append("2013_08_negative")

In [120]:
summary.append(response.choices[0].message.content)

In [121]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  8 non-null      object
 1   summary  8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [122]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...
6,2013_07_negative,The sentences discuss various opinions and exp...
7,2013_08_negative,The text discusses various opinions and experi...


In [123]:
response

<OpenAIObject chat.completion id=chatcmpl-AUeiHTYuCLi0x0V2HP9mLJ6q1lGPg at 0x249ece9b860> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various opinions and experiences regarding vaccinations in Brazil, including pain and discomfort associated with certain vaccines, especially the HPV and tetanus vaccines. Many express frustration with the healthcare system's inefficiencies, lack of available vaccines, and the feeling of being forced to vaccinate. There's also mention of new vaccines being developed or tested, including one against cancer and another for HIV, alongside criticism of government policies related to vaccination campaigns. Overall, the sentiment reflects a mix of fear, skepticism, and dissatisfaction with vaccination processes.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869705,
  "id": "chatcmpl-AUeiHTYuCLi0x0V2HP9

## September

In [124]:
# Get monthly data
df_2013_09_neg = pd.read_csv('../data/local/df_neg_2013-09.csv').reset_index()
df_2013_09_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   914 non-null    int64  
 1   LocationCode            914 non-null    object 
 2   LocationName            914 non-null    object 
 3   StatesCode              914 non-null    object 
 4   ...4                    914 non-null    int64  
 5   id                      914 non-null    float64
 6   text                    914 non-null    object 
 7   label                   914 non-null    object 
 8   in_reply_to_status_id   77 non-null     float64
 9   in_reply_to_user_id     96 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       143 non-null    float64
 13  retweeted_status_id     143 non-null    float64
 14  created_at              914 non-null    ob

### Representative sample based on week and location

In [125]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_09_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_09_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w35,Amazonas,1,0.001094
1,2013-w35,Bahia,4,0.004376
2,2013-w35,Goias,2,0.002188
3,2013-w35,Maranhao,1,0.001094
4,2013-w35,Mato Grosso Do Sul,1,0.001094
...,...,...,...,...
82,2013-w39,Paraiba,18,0.019694
83,2013-w39,Parana,7,0.007659
84,2013-w39,Rio Grande Do Sul,23,0.025164
85,2013-w39,Santa Catarina,6,0.006565


In [126]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w35,Amazonas,1,0.001094,2
1,2013-w35,Bahia,4,0.004376,10
2,2013-w35,Goias,2,0.002188,5
3,2013-w35,Maranhao,1,0.001094,2
4,2013-w35,Mato Grosso Do Sul,1,0.001094,2
...,...,...,...,...,...
82,2013-w39,Paraiba,18,0.019694,48
83,2013-w39,Parana,7,0.007659,18
84,2013-w39,Rio Grande Do Sul,23,0.025164,61
85,2013-w39,Santa Catarina,6,0.006565,16


In [127]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_09_neg_sample_size = pd.merge(df_2013_09_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [128]:
# Step 6: Perform the stratified sampling
#df_2013_09_neg_sample = df_2013_09_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_09_neg_sample = df_2013_09_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_09_neg_sample)} rows.")


Sampled 914 rows.


In [129]:
text_gpt_2013_09_neg = '\n'.join(df_2013_09_neg_sample['text'].astype(str))

### Prompts and loops

In [130]:
prompt_9 = prompt_raw.replace('TWEETS', text_gpt_2013_09_neg)

In [131]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_9}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [132]:
response.choices[0].message.content

"The text reflects a mix of frustrations and humorous sentiments regarding vaccinations. Many express discomfort, pain, or fear associated with getting vaccinated, often highlighting the negative side effects like soreness and illness post-vaccination. There are also references to the necessity of vaccinations for pets, with some lamenting the challenges of taking animals to receive their shots. Additionally, some comments criticize the healthcare system's conditions and raise concerns about misinformation surrounding vaccines. Overall, the sentiment is a blend of annoyance, fear, and sarcasm towards the vaccination process and its implications."

In [133]:
#summary = []
#dataset = []

In [134]:
dataset.append("2013_09_negative")

In [135]:
summary.append(response.choices[0].message.content)

In [136]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  9 non-null      object
 1   summary  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [137]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...
6,2013_07_negative,The sentences discuss various opinions and exp...
7,2013_08_negative,The text discusses various opinions and experi...
8,2013_09_negative,The text reflects a mix of frustrations and hu...


In [138]:
response

<OpenAIObject chat.completion id=chatcmpl-AUeiLqlqBnjwD8U5iYCPjTzGUifi6 at 0x249ecf6cef0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text reflects a mix of frustrations and humorous sentiments regarding vaccinations. Many express discomfort, pain, or fear associated with getting vaccinated, often highlighting the negative side effects like soreness and illness post-vaccination. There are also references to the necessity of vaccinations for pets, with some lamenting the challenges of taking animals to receive their shots. Additionally, some comments criticize the healthcare system's conditions and raise concerns about misinformation surrounding vaccines. Overall, the sentiment is a blend of annoyance, fear, and sarcasm towards the vaccination process and its implications.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869709,
  "id": "chatcmpl

## October

In [139]:
# Get monthly data
df_2013_10_neg = pd.read_csv('../data/local/df_neg_2013-10.csv').reset_index()
df_2013_10_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   812 non-null    int64  
 1   LocationCode            812 non-null    object 
 2   LocationName            812 non-null    object 
 3   StatesCode              812 non-null    object 
 4   ...4                    812 non-null    int64  
 5   id                      812 non-null    float64
 6   text                    812 non-null    object 
 7   label                   812 non-null    object 
 8   in_reply_to_status_id   91 non-null     float64
 9   in_reply_to_user_id     97 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       67 non-null     float64
 13  retweeted_status_id     67 non-null     float64
 14  created_at              812 non-null    ob

### Representative sample based on week and location

In [140]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_10_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_10_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w40,Alagoas,1,0.001232
1,2013-w40,Amazonas,8,0.009852
2,2013-w40,Bahia,7,0.008621
3,2013-w40,Ceara,5,0.006158
4,2013-w40,Distrito Federal,12,0.014778
...,...,...,...,...
83,2013-w44,Rio Grande Do Sul,3,0.003695
84,2013-w44,Rondonia,1,0.001232
85,2013-w44,Roraima,1,0.001232
86,2013-w44,Santa Catarina,5,0.006158


In [141]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w40,Alagoas,1,0.001232,3
1,2013-w40,Amazonas,8,0.009852,24
2,2013-w40,Bahia,7,0.008621,21
3,2013-w40,Ceara,5,0.006158,15
4,2013-w40,Distrito Federal,12,0.014778,36
...,...,...,...,...,...
83,2013-w44,Rio Grande Do Sul,3,0.003695,9
84,2013-w44,Rondonia,1,0.001232,3
85,2013-w44,Roraima,1,0.001232,3
86,2013-w44,Santa Catarina,5,0.006158,15


In [142]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_10_neg_sample_size = pd.merge(df_2013_10_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [143]:
# Step 6: Perform the stratified sampling
#df_2013_10_neg_sample = df_2013_10_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
#    lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_10_neg_sample = df_2013_10_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_10_neg_sample)} rows.")


Sampled 812 rows.


In [144]:
text_gpt_2013_10_neg = '\n'.join(df_2013_10_neg_sample['text'].astype(str))

### Prompts and loops

In [145]:
prompt_10 = prompt_raw.replace('TWEETS', text_gpt_2013_10_neg)

In [146]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_10}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [147]:
response.choices[0].message.content

'The text consists of various comments and reactions regarding vaccinations, expressing a mix of fear, annoyance, and discomfort associated with the process. Many participants share their experiences of pain from recent vaccinations, express reluctance to receive vaccines, and criticize mandatory vaccination policies. There are also humorous and sarcastic remarks about the need for vaccines for both humans and pets. Themes of skepticism about vaccine efficacy and side effects, along with social commentary on vaccination policies, are prevalent throughout the messages.'

In [148]:
#summary = []
#dataset = []

In [149]:
dataset.append("2013_10_negative")

In [150]:
summary.append(response.choices[0].message.content)

In [151]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  10 non-null     object
 1   summary  10 non-null     object
dtypes: object(2)
memory usage: 288.0+ bytes


In [152]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...
6,2013_07_negative,The sentences discuss various opinions and exp...
7,2013_08_negative,The text discusses various opinions and experi...
8,2013_09_negative,The text reflects a mix of frustrations and hu...
9,2013_10_negative,The text consists of various comments and reac...


In [153]:
response

<OpenAIObject chat.completion id=chatcmpl-AUeiPSUiqa0tvFwDsxMYondt0wZpm at 0x249ed22abd0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text consists of various comments and reactions regarding vaccinations, expressing a mix of fear, annoyance, and discomfort associated with the process. Many participants share their experiences of pain from recent vaccinations, express reluctance to receive vaccines, and criticize mandatory vaccination policies. There are also humorous and sarcastic remarks about the need for vaccines for both humans and pets. Themes of skepticism about vaccine efficacy and side effects, along with social commentary on vaccination policies, are prevalent throughout the messages.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869713,
  "id": "chatcmpl-AUeiPSUiqa0tvFwDsxMYondt0wZpm",
  "model": "gpt-4o-mini-2024-07-18",
  "object

## November

In [154]:
# Get monthly data
df_2013_11_neg = pd.read_csv('../data/local/df_neg_2013-11.csv').reset_index()
df_2013_11_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   651 non-null    int64  
 1   LocationCode            651 non-null    object 
 2   LocationName            651 non-null    object 
 3   StatesCode              651 non-null    object 
 4   ...4                    651 non-null    int64  
 5   id                      651 non-null    float64
 6   text                    651 non-null    object 
 7   label                   651 non-null    object 
 8   in_reply_to_status_id   75 non-null     float64
 9   in_reply_to_user_id     82 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       89 non-null     float64
 13  retweeted_status_id     89 non-null     float64
 14  created_at              651 non-null    ob

### Representative sample based on week and location

In [155]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_11_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_11_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w44,Alagoas,1,0.001536
1,2013-w44,Amazonas,1,0.001536
2,2013-w44,Bahia,1,0.001536
3,2013-w44,Ceara,4,0.006144
4,2013-w44,Distrito Federal,1,0.001536
...,...,...,...,...
84,2013-w48,Pernambuco,3,0.004608
85,2013-w48,Rio Grande Do Sul,19,0.029186
86,2013-w48,Santa Catarina,2,0.003072
87,2013-w48,Sao Paulo,28,0.043011


In [156]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w44,Alagoas,1,0.001536,3
1,2013-w44,Amazonas,1,0.001536,3
2,2013-w44,Bahia,1,0.001536,3
3,2013-w44,Ceara,4,0.006144,15
4,2013-w44,Distrito Federal,1,0.001536,3
...,...,...,...,...,...
84,2013-w48,Pernambuco,3,0.004608,11
85,2013-w48,Rio Grande Do Sul,19,0.029186,71
86,2013-w48,Santa Catarina,2,0.003072,7
87,2013-w48,Sao Paulo,28,0.043011,105


In [157]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_11_neg_sample_size = pd.merge(df_2013_11_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [158]:
# Step 6: Perform the stratified sampling
#df_2013_11_neg_sample = df_2013_11_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_11_neg_sample = df_2013_11_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_11_neg_sample)} rows.")


Sampled 651 rows.


In [159]:
text_gpt_2013_11_neg = '\n'.join(df_2013_11_neg_sample['text'].astype(str))

### Prompts and loops

In [160]:
prompt_11 = prompt_raw.replace('TWEETS', text_gpt_2013_11_neg)

In [161]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_11}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [162]:
response.choices[0].message.content

"The text discusses various aspects of vaccination, including public sentiments about vaccine pain and fear, financial concerns for producers regarding the cost of vaccines, and controversies surrounding vaccines, such as alleged adverse effects from the HPV vaccine. Additionally, there are mentions of testing new vaccines on animals and the public's reaction to vaccination campaigns."

In [163]:
#summary = []
#dataset = []

In [164]:
dataset.append("2013_11_negative")

In [165]:
summary.append(response.choices[0].message.content)

In [166]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  11 non-null     object
 1   summary  11 non-null     object
dtypes: object(2)
memory usage: 304.0+ bytes


In [167]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...
6,2013_07_negative,The sentences discuss various opinions and exp...
7,2013_08_negative,The text discusses various opinions and experi...
8,2013_09_negative,The text reflects a mix of frustrations and hu...
9,2013_10_negative,The text consists of various comments and reac...


In [168]:
response

<OpenAIObject chat.completion id=chatcmpl-AUeiTCUbAwKCpfcnHqT6bw4LWQT9h at 0x249ed105a40> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses various aspects of vaccination, including public sentiments about vaccine pain and fear, financial concerns for producers regarding the cost of vaccines, and controversies surrounding vaccines, such as alleged adverse effects from the HPV vaccine. Additionally, there are mentions of testing new vaccines on animals and the public's reaction to vaccination campaigns.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869717,
  "id": "chatcmpl-AUeiTCUbAwKCpfcnHqT6bw4LWQT9h",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_0ba0d124f1",
  "usage": {
    "completion_tokens": 62,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "a

## December

In [169]:
# Get monthly data
df_2013_12_neg = pd.read_csv('../data/local/df_neg_2013-12.csv').reset_index()
df_2013_12_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index                   646 non-null    int64  
 1   LocationCode            646 non-null    object 
 2   LocationName            646 non-null    object 
 3   StatesCode              646 non-null    object 
 4   ...4                    646 non-null    int64  
 5   id                      646 non-null    float64
 6   text                    646 non-null    object 
 7   label                   646 non-null    object 
 8   in_reply_to_status_id   43 non-null     float64
 9   in_reply_to_user_id     50 non-null     float64
 10  quoted_user_id          0 non-null      float64
 11  quoted_status_id        0 non-null      float64
 12  retweeted_user_id       146 non-null    float64
 13  retweeted_status_id     146 non-null    float64
 14  created_at              646 non-null    ob

### Representative sample based on week and location

In [170]:
# Step 1: Calculate the size of each group (year_week, LocationName)
group_sizes = df_2013_12_neg.groupby(['year_week', 'LocationName']).size().reset_index(name='group_size')

# Step 2: Calculate the proportion of each group in the dataset
total_rows = len(df_2013_12_neg)
group_sizes['proportion'] = group_sizes['group_size'] / total_rows
group_sizes

,year_week,LocationName,group_size,proportion
0,2013-w48,Acre,1,0.001548
1,2013-w48,Amazonas,1,0.001548
2,2013-w48,Bahia,1,0.001548
3,2013-w48,Distrito Federal,1,0.001548
4,2013-w48,Mato Grosso Do Sul,2,0.003096
...,...,...,...,...
75,2013-w52,Roraima,2,0.003096
76,2013-w52,Santa Catarina,3,0.004644
77,2013-w52,Sao Paulo,28,0.043344
78,2013-w53,Pernambuco,1,0.001548


In [171]:
# Step 3: Define the target sample size
target_sample_size = 2450

# Step 4: Calculate the sample size for each group (based on its proportion)
group_sizes['sample_size'] = np.floor(group_sizes['proportion'] * target_sample_size).astype(int)
group_sizes

,year_week,LocationName,group_size,proportion,sample_size
0,2013-w48,Acre,1,0.001548,3
1,2013-w48,Amazonas,1,0.001548,3
2,2013-w48,Bahia,1,0.001548,3
3,2013-w48,Distrito Federal,1,0.001548,3
4,2013-w48,Mato Grosso Do Sul,2,0.003096,7
...,...,...,...,...,...
75,2013-w52,Roraima,2,0.003096,7
76,2013-w52,Santa Catarina,3,0.004644,11
77,2013-w52,Sao Paulo,28,0.043344,106
78,2013-w53,Pernambuco,1,0.001548,3


In [172]:
# Step 5: Merge this sample size information back to the original DataFrame
df_2013_12_neg_sample_size = pd.merge(df_2013_12_neg, group_sizes[['year_week', 'LocationName', 'sample_size']], 
                               on=['year_week', 'LocationName'], how='left')

In [173]:
# Step 6: Perform the stratified sampling
#df_2013_12_neg_sample = df_2013_12_neg_sample_size.groupby(['year_week', 'LocationName']).apply(
 #   lambda group: group.sample(n=group['sample_size'].iloc[0], random_state=42)
#).reset_index(drop=True)
df_2013_12_neg_sample = df_2013_12_neg
# Step 7: Check the result
print(f"Sampled {len(df_2013_12_neg_sample)} rows.")


Sampled 646 rows.


In [174]:
text_gpt_2013_12_neg = '\n'.join(df_2013_12_neg_sample['text'].astype(str))

### Prompts and loops

In [175]:
prompt_12 = prompt_raw.replace('TWEETS', text_gpt_2013_12_neg)

In [176]:
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",  
    messages = [
        {"role": "user", "content": prompt_12}
    ],
    max_tokens=400,  # Maximum number of tokens in the response
    temperature=0.7  # Controls the randomness of the output
)

In [177]:
response.choices[0].message.content

'The text discusses the high costs of vaccines, particularly highlighting the lack of access and availability in Brazil. It mentions issues related to the effectiveness and side effects of vaccines, including pain and discomfort following administration. There are also references to tragic outcomes in China linked to vaccinations, as well as societal attitudes and controversies surrounding vaccines.'

In [178]:
#summary = []
#dataset = []

In [179]:
dataset.append("2013_12_negative")

In [180]:
summary.append(response.choices[0].message.content)

In [181]:
df_2013_neg = pd.DataFrame(list(zip(dataset, summary)),
                            columns = ['dataset', 'summary'])
df_2013_neg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   dataset  12 non-null     object
 1   summary  12 non-null     object
dtypes: object(2)
memory usage: 320.0+ bytes


In [182]:
df_2013_neg.tail(14)

,dataset,summary
0,2013_01_negative,The texts discuss various vaccine-related issu...
1,2013_02_negative,The text discusses various experiences and opi...
2,2013_03_negative,The text expresses frustration and concerns ab...
3,2013_04_negative,"The texts express a mix of humor, fear, and fr..."
4,2013_05_negative,The sentences express a widespread dislike and...
5,2013_06_negative,The text primarily discusses the controversial...
6,2013_07_negative,The sentences discuss various opinions and exp...
7,2013_08_negative,The text discusses various opinions and experi...
8,2013_09_negative,The text reflects a mix of frustrations and hu...
9,2013_10_negative,The text consists of various comments and reac...


In [183]:
response

<OpenAIObject chat.completion id=chatcmpl-AUeiWqfizXROGGPNB0YnVmQagjGr1 at 0x249ed240770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The text discusses the high costs of vaccines, particularly highlighting the lack of access and availability in Brazil. It mentions issues related to the effectiveness and side effects of vaccines, including pain and discomfort following administration. There are also references to tragic outcomes in China linked to vaccinations, as well as societal attitudes and controversies surrounding vaccines.",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1731869720,
  "id": "chatcmpl-AUeiWqfizXROGGPNB0YnVmQagjGr1",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "system_fingerprint": "fp_9b78b61c52",
  "usage": {
    "completion_tokens": 63,
    "completion_tokens_details": {
      "accepted_prediction_token

## All 2013 negative

In [184]:
#df_2013_neg['dataset'] = df_2013_neg['dataset'].str.replace('negitive','negative')

In [185]:
df_2013_neg.to_csv('../data/summary_2013_negative_tweets.csv')